In [1]:
!conda install -c anaconda beautifulsoup4 --yes
# import libraries
from bs4 import BeautifulSoup
import urllib.request
import csv
import pandas as pd

Solving environment: done

# All requested packages already installed.



In [2]:
# specify the url
urlpage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(urlpage)
# parse the html using beautiful soup and store in variable 
bsObj = BeautifulSoup(page, 'html.parser')
# find results within table
tab = bsObj.findAll("tbody")
data_list=[]
for ligne in tab[0].findAll("tr"):
    l=[]
    for cellule in ligne.findAll("td"):
        l.append(((str(cellule).replace("<td>","")).replace("</td>","")).replace("\n",""))
    data_list.append(l)
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
data=pd.DataFrame(data=data_list,columns=['Postalcode','Borough','Neighborhood'])
data.drop([0], inplace=True)

In [3]:
#Ignore cells with a borough that is Not assigned.
data.drop(data[data.Borough =='Not assigned'].index, inplace=True)

In [4]:
#More than one neighborhood can exist in one postal code area.
data['Neighborhood'] = data.Neighborhood.str.replace('/',',')

In [5]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for row in data.iterrows():
    if row[1]['Neighborhood']=="":
        row[1]['Neighborhood']=row[1]['Borough']

In [69]:
data  

,Postalcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park , Harbourfront"
6,M6A,North York,"Lawrence Manor , Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
161,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
166,M4Y,Downtown Toronto,Church and Wellesley
169,M7Y,East Toronto,Business reply mail Processing Centre
170,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [6]:
data.shape

(103, 3)

In [51]:
geo_data=pd.read_csv('Geospatial_Coordinates.csv')
geo_data.rename(columns={"Postal Code": 'Postalcode'}, inplace=True)
geo_data

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [78]:
data2= data.assign(Latitude="", Longitude="")
x=geo_data[geo_data['Postalcode']=="M9W"][['Latitude','Longitude']].values[0]
x[0]

43.706748299999994

In [83]:
#import geocoder # import geocoder
import numpy as np
for row in data2.iterrows():
    p=row[1]['Postalcode']
    x=geo_data[geo_data['Postalcode']==p][['Latitude','Longitude']].values[0]
    if x == []:
        print(p)
    else:
        row[1]['Latitude']=x[0]
        row[1]['Longitude']=x[1]
data2  

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


,Postalcode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.7533,-79.3297
4,M4A,North York,Victoria Village,43.7259,-79.3156
5,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6543,-79.3606
6,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7185,-79.4648
7,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6623,-79.3895
...,...,...,...,...,...
161,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.6537,-79.5069
166,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
169,M7Y,East Toronto,Business reply mail Processing Centre,43.6627,-79.3216
170,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.6363,-79.4985
